In [3]:
import csv
import re

log_dir = "./level_base_log_data"
log_file_path = f"{log_dir}/h1_fillrandom_cs0_size256_base67108864.log"
csv_output_path = "./output.csv"

# 데이터 저장용
ops_list = []
stats_dict = {}

with open(log_file_path, "r") as f:
    for line in f:
        line = line.strip()

        # "finished XXX ops" 감지
        match_ops = re.match(r"\.\.\. finished (\d+) ops", line)
        if match_ops:
            ops_list.append(int(match_ops.group(1)))

        # "rocksdb.XXX COUNT : N" 감지
        match_stat = re.match(r"(rocksdb\.[\w\.]+) COUNT\s*:\s*(\d+)", line)
        if match_stat:
            key = match_stat.group(1)
            value = int(match_stat.group(2))
            stats_dict[key] = value

# 결과를 CSV로 저장
with open(csv_output_path, mode="w", newline='') as csvfile:
    writer = csv.writer(csvfile)

    # 첫 번째 시트: ops 로그
    writer.writerow(["Ops Sequence"])
    for ops in ops_list:
        writer.writerow([ops])

    # 빈 줄로 구분
    writer.writerow([])

    # 두 번째 시트: RocksDB stats
    writer.writerow(["Stat Name", "Count"])
    for key, value in stats_dict.items():
        writer.writerow([key, value])

print("CSV 변환이 완료되었습니다.")

CSV 변환이 완료되었습니다.


In [1]:
import csv
import re
import os

log_dir = "./level_base_log_data"
log_file_name = "h1_fillrandom_cs0_size256_base67108864.log"
log_file_path = os.path.join(log_dir, log_file_name)
csv_output_path = "./output.csv"

# 정규표현식을 이용해 파일 이름에서 정보 추출
filename_pattern = r"h1_(\w+)_cs(\d+)_size(\d+)_base(\d+)\.log"
match = re.match(filename_pattern, log_file_name)

if match:
    workload = match.group(1)
    compaction_style = int(match.group(2))
    value_size = int(match.group(3))
    level_base = int(match.group(4))
else:
    raise ValueError("파일 이름에서 필요한 정보를 추출할 수 없습니다.")

# 데이터 저장용
ops_list = []
stats_dict = {}

with open(log_file_path, "r") as f:
    for line in f:
        line = line.strip()

        # "finished XXX ops" 감지
        match_ops = re.match(r"\.\.\. finished (\d+) ops", line)
        if match_ops:
            ops_list.append(int(match_ops.group(1)))

        # "rocksdb.XXX COUNT : N" 감지
        match_stat = re.match(r"(rocksdb\.[\w\.]+) COUNT\s*:\s*(\d+)", line)
        if match_stat:
            key = match_stat.group(1)
            value = int(match_stat.group(2))
            stats_dict[key] = value

# 결과를 CSV로 저장
with open(csv_output_path, mode="w", newline='') as csvfile:
    writer = csv.writer(csvfile)

    # 헤더
    writer.writerow([
        "Type", "Name/Value", "Count", "Workload", 
        "Compaction Style", "Value Size", "Level Base"
    ])

    # ops 리스트 저장
    for idx, ops in enumerate(ops_list):
        writer.writerow([
            "Ops", f"Sequence {idx+1}", ops, workload, 
            compaction_style, value_size, level_base
        ])

    # RocksDB stat 저장
    for key, value in stats_dict.items():
        writer.writerow([
            "Stat", key, value, workload, 
            compaction_style, value_size, level_base
        ])

print("CSV 변환이 완료되었습니다.")


CSV 변환이 완료되었습니다.


In [4]:
import pandas as pd

# CSV 파일을 읽어 DataFrame으로 변환
df = pd.read_csv(csv_output_path)

df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 39, saw 2
